In [ ]:
!pip install gradio fairlearn==0.8.0 alibi==0.7.0 nltk
!pip install tensorflow

In [ ]:
import random
import json
import pickle
import numpy as np
from tensorflow.keras.models import load_model
from nltk.stem import WordNetLemmatizer
import nltk
import re

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('punkt_tab')

lemmatizer = WordNetLemmatizer()

with open('/content/intents.json') as file:
    intents_data = json.load(file)

model = load_model('/content/chatbot_model.h5')
with open('/content/words.pkl', 'rb') as f:
    words = pickle.load(f)
with open('/content/classes.pkl', 'rb') as f:
    classes = pickle.load(f)

def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

def bag_of_words(sentence_words, words, show_details=True):
    # Initialize the bag list here
    bag = [0] * len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence):
    sentence_words = clean_up_sentence(sentence)
    bag = bag_of_words(sentence_words, words)
    res = model.predict(np.array([bag]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def get_response(ints, intents_json):
    try:
        tag = ints[0]['intent']
        list_of_intents = intents_json['intents']
        for i in list_of_intents:
            if(i['tag']== tag):
                result = random.choice(i['responses'])
                break
        return result
    except IndexError:
        return "I'm sorry, I didn't understand that."

def chatbot_response(user_input):
    ints = predict_class(user_input)
    res = get_response(ints, intents_data)
    return res

while True:
    message = input("You: ")
    if message.lower() == "quit":
        break
    response = chatbot_response(message)
    print(f"Bot: {response}")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


You: hi
found in bag: hi
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
Bot: Hi! How can I assist you today?
You: covid precation
found in bag: covid
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Bot: To stay safe: wear a mask, maintain social distancing, wash your hands frequently, and get vaccinated.


KeyboardInterrupt: Interrupted by user

In [ ]:

import random
import json
import pickle
import numpy as np
from tensorflow.keras.models import load_model
from nltk.stem import WordNetLemmatizer
import gradio as gr
import pandas as pd
from fairlearn.metrics import MetricFrame, selection_rate, false_positive_rate, true_positive_rate
import matplotlib.pyplot as plt
import seaborn as sns
import io
import base64
import nltk

nltk.download('punkt')
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()

with open('/content/intents.json') as file:
    intents_data = json.load(file)

model = load_model('/content/chatbot_model.h5')
with open('/content/words.pkl', 'rb') as f:
    words = pickle.load(f)
with open('/content/classes.pkl', 'rb') as f:
    classes = pickle.load(f)

if 'appointment_booking' not in classes:
    classes.append('appointment_booking')
    with open('/content/classes.pkl', 'wb') as f:
        pickle.dump(classes, f)
    print("Added 'appointment_booking' to classes and updated classes.pkl")

def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

def bag_of_words(sentence, words, perturbed=False):
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                bag[i] = 1

    if perturbed:
        bag = np.array(bag) + np.random.normal(0, 0.1, size=len(bag))

    return np.array(bag)

def predict_class(sentence, bow=None):
    if bow is None:
        bow = bag_of_words(sentence, words)
    res = model.predict(np.array([bow]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return [{"intent": classes[r[0]], "probability": str(r[1])} for r in results]

def get_response(intent_list, intents_json):
    try:
        tag = intent_list[0]['intent']
        list_of_intents = intents_json['intents']
        for i in list_of_intents:
            if i['tag'] == tag:
                return random.choice(i['responses'])
    except IndexError:
        return "I'm sorry, I didn't understand that."


def chatbot_interface(user_input):
    intents = predict_class(user_input)
    response = get_response(intents, intents_data)
    return response

def model_robustness_dashboard(user_input):
    try:
        original_intent = predict_class(user_input)[0]['intent']
        perturbed_bow = bag_of_words(user_input, words, perturbed=True)
        perturbed_intent = predict_class(user_input, perturbed_bow)[0]['intent']
        robustness_result = f"Original Intent: {original_intent}, Perturbed Intent: {perturbed_intent}"
        robustness_result += "\nModel is robust to this perturbation." if original_intent == perturbed_intent else "\nModel is sensitive to this perturbation."
        return robustness_result
    except Exception as e:
        return f"Error in robustness analysis: {e}"

def model_confidence_dashboard(user_input):
    """Displays the model's confidence in its prediction with a bar graph."""
    intents = predict_class(user_input)
    if intents:

        plt.figure(figsize=(8, 4))
        plt.bar(x=[intent['intent'] for intent in intents], height=[float(intent['probability']) for intent in intents])
        plt.title('Model Confidence for Each Intent')
        plt.xlabel('Intent')
        plt.ylabel('Confidence')
        plt.ylim(0, 1)


        buf = io.BytesIO()
        plt.savefig(buf, format='png')
        buf.seek(0)
        image_data = base64.b64encode(buf.read()).decode('utf-8')
        plt.close()

        return f"<img src='data:image/png;base64,{image_data}'/>"
    else:
        return "Model is not confident about any intent."

def assess_model_fairness(X, y, sensitive_features):
    """Assesses model fairness with respect to gender using Fairlearn."""

    mf = MetricFrame(
        metrics={
            "selection_rate": selection_rate,
            "false_positive_rate": false_positive_rate,
            "true_positive_rate": true_positive_rate,
        },
        y_true=y,
        y_pred=y,
        sensitive_features=sensitive_features,
    )
    return mf.by_group, mf.overall


X = ["sample input 1", "sample input 2"]
y = [0, 1]
sensitive_features = ["male", "female"]
fairness_report = assess_model_fairness(X, y, sensitive_features)

def combined_function(user_input):
    """Combines all functions to handle user input."""
    chatbot_response = chatbot_interface(user_input)
    robustness_result = model_robustness_dashboard(user_input)
    confidence_result = model_confidence_dashboard(user_input)


    fairness_result = str(fairness_report[0]) + "\nOverall: " + str(fairness_report[1])


    return chatbot_response, robustness_result, confidence_result, fairness_result

iface = gr.Interface(
    fn=combined_function,
    inputs=gr.Textbox(lines=2, placeholder="Enter your message here..."),
    outputs=[
        "text",
        gr.Textbox(label="Model Robustness"),
        gr.HTML(label="Model Confidence"),
        gr.Textbox(label="Fairness Report")
    ],
    title="Chatbot with Responsible AI Dashboards",
    description="Chat and explore model insights."
)

iface.launch()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://305f67bf513a5916c4.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
